In [5]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('/dataset.csv')

# Step 1: Data Preprocessing

# Combine date and time into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Encode TrafficConditionNearby
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

# Encode VehicleType weights
vehicle_weight = {'bike': 0.5, 'car': 1.0, 'truck': 1.5}
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_weight)

# Sort by parking lot and time for consistent simulation
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp'])

# Step 2: Model 1 - Baseline Linear Pricing Model

# Initialize base price and alpha
base_price = 10
alpha = 2
min_price, max_price = 5, 20

# Create a dictionary to store latest price for each parking lot
latest_price = {lot: base_price for lot in df['SystemCodeNumber'].unique()}
prices = []

# Simulate dynamic pricing for each record
for _, row in df.iterrows():
    lot = row['SystemCodeNumber']
    occupancy = row['Occupancy']
    capacity = row['Capacity']
    current_price = latest_price[lot]

    # Linear price update
    updated_price = current_price + alpha * (occupancy / capacity)
    updated_price = np.clip(updated_price, min_price, max_price)

    # Store and update
    prices.append(updated_price)
    latest_price[lot] = updated_price

# Add to DataFrame
df['Price_Model1'] = prices

# Preview
print(df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'Capacity', 'Price_Model1']].head(10))

##Model 2
# Load dataset
df = pd.read_csv('/dataset.csv')

# Step 1: Data Preprocessing

# Combine date and time into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Encode TrafficConditionNearby
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)

# Encode VehicleType weights
vehicle_weight = {'bike': 0.5, 'car': 1.0, 'truck': 1.5}
df['VehicleTypeWeight'] = df['VehicleType'].map(vehicle_weight)

# Sort by parking lot and time
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp'])

# Step 2: Model 1 - Baseline Linear Pricing Model

# Initialize base price and alpha
base_price = 10
alpha = 2
min_price, max_price = 5, 20

# Create a dictionary to store latest price for each parking lot
latest_price = {lot: base_price for lot in df['SystemCodeNumber'].unique()}
prices = []

# Simulate dynamic pricing for each record
for _, row in df.iterrows():
    lot = row['SystemCodeNumber']
    occupancy = row['Occupancy']
    capacity = row['Capacity']
    current_price = latest_price[lot]

    # Linear price update
    updated_price = current_price + alpha * (occupancy / capacity)
    updated_price = np.clip(updated_price, min_price, max_price)

    prices.append(updated_price)
    latest_price[lot] = updated_price

df['Price_Model1'] = prices

# Step 3: Model 2 - Demand-Based Pricing Model

# Define coefficients for demand function
alpha_d, beta_d, gamma_d, delta_d, epsilon_d = 1.0, 0.5, 0.3, 2.0, 1.0
lambda_d = 0.8  # Scaling factor for price adjustment

# Calculate raw demand score
df['RawDemand'] = (
    alpha_d * (df['Occupancy'] / df['Capacity']) +
    beta_d * df['QueueLength'] -
    gamma_d * df['TrafficLevel'] +
    delta_d * df['IsSpecialDay'] +
    epsilon_d * df['VehicleTypeWeight']
)

# Normalize demand
demand_min = df['RawDemand'].min()
demand_max = df['RawDemand'].max()
df['NormalizedDemand'] = (df['RawDemand'] - demand_min) / (demand_max - demand_min)

# Calculate price from normalized demand
df['Price_Model2'] = base_price * (1 + lambda_d * df['NormalizedDemand'])
df['Price_Model2'] = df['Price_Model2'].clip(lower=min_price, upper=max_price)

# Preview final results
print(df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'QueueLength', 'TrafficLevel',
          'IsSpecialDay', 'VehicleTypeWeight', 'NormalizedDemand',
          'Price_Model1', 'Price_Model2']].head(10))

  SystemCodeNumber           Timestamp  Occupancy  Capacity  Price_Model1
0      BHMBCCMKT01 2016-10-04 07:59:00         61       577     10.211438
1      BHMBCCMKT01 2016-10-04 08:25:00         64       577     10.433276
2      BHMBCCMKT01 2016-10-04 08:59:00         80       577     10.710572
3      BHMBCCMKT01 2016-10-04 09:32:00        107       577     11.081456
4      BHMBCCMKT01 2016-10-04 09:59:00        150       577     11.601386
5      BHMBCCMKT01 2016-10-04 10:26:00        177       577     12.214905
6      BHMBCCMKT01 2016-10-04 10:59:00        219       577     12.974003
7      BHMBCCMKT01 2016-10-04 11:25:00        247       577     13.830156
8      BHMBCCMKT01 2016-10-04 11:59:00        259       577     14.727903
9      BHMBCCMKT01 2016-10-04 12:29:00        266       577     15.649913
  SystemCodeNumber           Timestamp  Occupancy  QueueLength  TrafficLevel  \
0      BHMBCCMKT01 2016-10-04 07:59:00         61            1           1.0   
1      BHMBCCMKT01 2016-10